In [14]:
import chromadb
#from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_nomic.embeddings import NomicEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import Ollama
from langchain.chains import RetrievalQA
#from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

# See full prompt at https://smith.langchain.com/hub/rlm/rag-prompt
#prompt = hub.pull("rlm/rag-prompt")

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# # Prompt
# prompt = PromptTemplate(
#     template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
#     Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
#     Use ten sentences maximum <|eot_id|><|start_header_id|>user<|end_header_id|>
#     Question: {question} 
#     Context: {context} 
#     Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#     input_variables=["question", "document"],
# )


# Connect to the ChromaDB client
client = chromadb.Client()  # Replace with your ChromaDB path

# Create embeddings
embedding_model = NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local")

# Load the Chroma collection
collection_name = "America"

# Specify the directory where the Chroma vector data is stored
PERSIST_DIRECTORY = "./persist_nov_22_b"  # Update with your directory path

# Connect to your Chroma database (vector store) with persistence
vectorstore = Chroma(
    collection_name=collection_name, 
    embedding_function=embedding_model,
    persist_directory=PERSIST_DIRECTORY  # Specify persistent directory
)

# # Create the LLM
llm = Ollama(model="llama3.1")

# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {
        "context": vectorstore.as_retriever(), #| format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

qa_chain.invoke("what does it say about Spain?")




'According to the text, one of the intentions behind preparing the Gutiérrez map was "to define clearly Spain\'s America for the other European powers who might have designs on the region." This suggests that the map was intended to assert Spanish claims and territory in the Americas.'